In [1]:
#Imports neccessary packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import econtools.metrics as mt
import statistics
from prettytable import PrettyTable

#Reads *.dta files into pandas
table1 = pd.read_stata("C:\\Users\\littl\\OneDrive\\Desktop\\ReplicationsModern\\rdz027_supplementary_data\\Supplementary\\Dataforweb\\city_dataset.dta")
table2 = pd.read_stata("C:\\Users\\littl\\OneDrive\\Desktop\\ReplicationsModern\\rdz027_supplementary_data\\Supplementary\\Dataforweb\\MSA_dataset.dta")
table3 = pd.read_stata("C:\\Users\\littl\\OneDrive\\Desktop\\ReplicationsModern\\rdz027_supplementary_data\\Supplementary\\Dataforweb\\CD_panel_dataset.dta")

In [2]:
%%capture --no-stdout

#Converts strings to numerics
table1.replace(('Yes', 'No'), (1, 0), inplace=True)

#This starts the years at zero instead of i.e. 1980
table1["year"] = pd.to_numeric(table1['year'],downcast="integer")

#Factorizes qualitative data into quantitative values
table1["city"] = pd.factorize(table1["city"])[0]
table1["metarea"] = pd.factorize(table1["metarea"])[0]
table1["stateicp"] = pd.factorize(table1["stateicp"])[0]

#Creates dummy variables for stateyear codes
istateyear_code = pd.get_dummies(table1['stateyear'],prefix="stateyear",dtype=float)
for i in range(0,len(istateyear_code.columns)):
    string = istateyear_code.columns[i]
    table1[string] = istateyear_code[string]
    
#Creates dummy variables for city codes
icity_code = pd.get_dummies(table1['city'],prefix="city",dtype=float)
for i in range(0,len(icity_code.columns)):
    string = icity_code.columns[i]
    table1[string] = icity_code[string]

In [3]:
%%capture --no-stdout

#Converts strings to numerics
table2.replace(('Yes', 'No'), (1, 0), inplace=True)

#This starts the years at zero instead of i.e. 1980
table2["year"] = pd.to_numeric(table2['year'],downcast="integer")

#Factorizes qualitative data into quantitative values
table2["metarea"] = pd.factorize(table2["metarea"])[0]
table2["stateicp"] = pd.factorize(table2["stateicp"])[0]

#Creates dummy variables for stateyear codes
istateyear2_code = pd.get_dummies(table2['stateyear'],prefix="stateyear",dtype=float)
for i in range(0,len(istateyear_code.columns)):
    string = istateyear2_code.columns[i]
    table2[string] = istateyear2_code[string]

In [4]:
%%capture --no-stdout

stateyeardummy = []
for element in table1.columns:
    if element.startswith("stateyear_"):
        stateyeardummy.append(element)
citydummy = []
for element in table1.columns:
    if element.startswith("city_"):
        citydummy.append(element)

stateyeardummy.pop(0)
citydummy.pop(0)

stateyear2dummy = []
for element in table2.columns:
    if element.startswith("stateyear_"):
        stateyear2dummy.append(element)
        
stateyear2dummy.pop(0)

# Add intercept to the independent variables
cons = []
for i in range(0, len(table1)):
    cons.append(1)
table1["const"] = cons

cons = []
for i in range(0, len(table2)):
    cons.append(1)
table2["const"] = cons

In [5]:
%%capture --no-stdout

n=table1
y = ["fr_recEU"]
x = ['ZEUMSA_predpop','const']
x = x+stateyeardummy
while 0 < 1:
        try:
            result1 = mt.reg(n, y, x, check_colinear=True, fe_name='city',cluster='metarea')
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            if len(varlist) == 0:
                result1 = mt.reg(n, y, x, fe_name='city',cluster='metarea')
                break
y = ["recentEU_pop900"]
x = ['ZMSAEU_pop900','const']
x = x+stateyeardummy
while 0 < 1:
        try:
            result2 = mt.reg(n, y, x, check_colinear=True, fe_name='city',cluster='metarea')
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            if len(varlist) == 0:
                result2 = mt.reg(n, y, x, fe_name='city',cluster='metarea')
                break
y = ["recentEU_predpop"]
x = ['ZEUMSA_predpop','const']
x = x+stateyeardummy
while 0 < 1:
        try:
            result3 = mt.reg(n, y, x, check_colinear=True, fe_name='city',cluster='metarea')
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            if len(varlist) == 0:
                result3 = mt.reg(n, y, x, fe_name='city',cluster='metarea')
                break
n = table2
y = ["recentEU_predpop"]
x = ['ZEUMSA_predpop']
x = x+stateyear2dummy

while 0 < 1:
        try:
            result4 = mt.reg(n, y, x, check_colinear=True, fe_name='metarea',cluster='metarea')
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            if len(varlist) == 0:
                result4 = mt.reg(n, y, x, fe_name='metarea',cluster='metarea')
                break
n = table1
y = ["recentEU_predpop"]
x = ['ZEUMSA_predpop','const']
l_citypop900a = []
l_imm900_Ya = []
for element in n.columns:
    if element.startswith("l_citypop900_Y"):
        l_citypop900a.append(element)
    if element.startswith("l_imm900_Y"):
        l_imm900_Ya.append(element)
x = x+stateyear2dummy+l_citypop900a+l_imm900_Ya
while 0 < 1:
        try:
            result5 = mt.reg(n, y, x, check_colinear=True, fe_name='city',cluster='metarea')
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            if len(varlist) == 0:
                result5 = mt.reg(n, y, x, fe_name='city',cluster='metarea')
                break
x = ['ZEUMSA_predpop','const']
l_value_added_est04_Ya = []
for element in n.columns:
    if element.startswith("l_value_added_est04_Y"):
        l_value_added_est04_Ya.append(element)
x = x+stateyear2dummy+l_value_added_est04_Ya
while 0 < 1:
        try:
            result6 = mt.reg(n, y, x, check_colinear=True, fe_name='city',cluster='metarea')
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            if len(varlist) == 0:
                result6 = mt.reg(n, y, x, fe_name='city',cluster='metarea')
                break

In [6]:
coeffs = ["Z"]
coeffs.append(format(round(result1.summary.coeff.ZEUMSA_predpop,3), '.3f'))
coeffs.append(format(round(result2.summary.coeff.ZMSAEU_pop900,3), '.3f'))
coeffs.append(format(round(result3.summary.coeff.ZEUMSA_predpop,3), '.3f'))
coeffs.append(format(round(result4.summary.coeff.ZEUMSA_predpop,3), '.3f'))
coeffs.append(format(round(result5.summary.coeff.ZEUMSA_predpop,3), '.3f'))
coeffs.append(format(round(result6.summary.coeff.ZEUMSA_predpop,3), '.3f'))

se = [""]
se.append("({})".format(round(result1.summary.se.ZEUMSA_predpop,3), '.3f'))
se.append("({})".format(round(result2.summary.se.ZMSAEU_pop900,3), '.3f'))
se.append("({})".format(round(result3.summary.se.ZEUMSA_predpop,3), '.3f'))
se.append("({})".format(round(result4.summary.se.ZEUMSA_predpop,3), '.3f'))
se.append("({})".format(round(result5.summary.se.ZEUMSA_predpop,3), '.3f'))
se.append("({})".format(round(result6.summary.se.ZEUMSA_predpop,3), '.3f'))

pop1900 = ["1900 population","","X","","","",""]
predpop = ["Predicted population","","","X","","",""]
msa = ["MSA analysis","","","","X","",""]
log1900 = ["Year by 1900 log","","","","","City and\nimm pop","Value added\nmanuf."]

f = ["F-stat"]
f.append(format(round(result1.Ftest(['ZEUMSA_predpop'])[0],1), '.1f'))
f.append(format(round(result2.Ftest(['ZMSAEU_pop900'])[0],1), '.1f'))
f.append(format(round(result3.Ftest(['ZEUMSA_predpop'])[0],1), '.1f'))
f.append(format(round(result4.Ftest(['ZEUMSA_predpop'])[0],2), '.2f'))
f.append(format(round(result5.Ftest(['ZEUMSA_predpop'])[0],2), '.2f'))
f.append(format(round(result6.Ftest(['ZEUMSA_predpop'])[0],1), '.1f'))

cities = ["Cities"]
cities.append(result1.fe_count)
cities.append(result2.fe_count)
cities.append(result3.fe_count)
cities.append(result4.fe_count)
cities.append(result5.fe_count)
cities.append(result6.fe_count)

nobs = ["Observations"]
nobs.append(result1.N)
nobs.append(result2.N)
nobs.append(result3.N)
nobs.append(result4.N)
nobs.append(result5.N)
nobs.append(result6.N)

In [7]:
regs = [result1,result2,result3,result4,result5,result6]
stars = []
for i in regs:
    if i.pt[0] < 0.01:
        stars.append("***")
    elif i.pt[0] < 0.05:
        stars.append("**")
    elif i.pt[0] < 0.1:
        stars.append("*")
for i in range(1,len(coeffs)):
    coeffs[i] = "{}{}".format(coeffs[i],stars[0])

In [8]:
mean = []
median = []
stdev = []
minval = []
maxval = []
obs = []
for i in (table1.fr_immEU, table1.fr_recEU, table1.recentEU_pop900, table1.citypop, table1.employed, table1.l_occscore, table1.value_added_manuf_est, table1.estab_size, table1.property_tax_rate, table1.expenses_pc, table2.avg_pres_dem, table3.dim1):
    filtered = filter(lambda x: not np.isnan(x), i)
    s = sum(filtered)
    filtered = filter(lambda x: not np.isnan(x), i)
    l = len(list((filtered)))
    mean.append(format(round(s/l,3), '.3f'))
    filtered = filter(lambda x: not np.isnan(x), i)
    median.append(format(round(np.median(list(filtered)),3), '.3f'))
    filtered = filter(lambda x: not np.isnan(x), i)
    stdev.append(format(round(np.std(list(filtered)),3), '.3f'))
    filtered = filter(lambda x: not np.isnan(x), i)
    minval.append(format(round(min(list(filtered)),3), '.3f')) 
    filtered = filter(lambda x: not np.isnan(x), i)
    maxval.append(format(round(max(list(filtered)),3), '.3f'))
    filtered = filter(lambda x: not np.isnan(x), i)
    obs.append(len(list(filtered)))
    
mean[3] = float(mean[3])/1000
median[3] = float(median[3])/1000
stdev[3] = float(stdev[3])/1000
minval[3] = float(minval[3])/1000
maxval[3] = float(maxval[3])/1000

In [9]:
mean.insert(8,"")
median.insert(8,"")
stdev.insert(8,"")
minval.insert(8,"")
maxval.insert(8,"")
obs.insert(8,"")
mean.insert(4,"")
median.insert(4,"")
stdev.insert(4,"")
minval.insert(4,"")
maxval.insert(4,"")
obs.insert(4,"")
mean.insert(0,"")
median.insert(0,"")
stdev.insert(0,"")
minval.insert(0,"")
maxval.insert(0,"")
obs.insert(0,"")

In [10]:
mean[4] = round(float(mean[4]),1)
mean[8] = round(float(mean[8]),2)
mean[9] = round(float(mean[9]),2)
mean[11] = round(float(mean[11]),2)
mean[12] = round(float(mean[12]),2)

median[8] = round(float(median[8]),2)
median[9] = round(float(median[9]),2)
median[11] = round(float(median[11]),2)
median[12] = round(float(median[12]),2)

stdev[4] = round(float(stdev[4]),1)
stdev[8] = format(round(float(stdev[8]),2), '.2f')
stdev[9] = round(float(stdev[9]),2)
stdev[11] = round(float(stdev[11]),2)

minval[4] = format(minval[4],'.2f')

maxval[4] = round(float(maxval[4]))
maxval[8] = round(float(maxval[8]),1)
maxval[9] = round(float(maxval[9]),1)
maxval[11] = round(float(maxval[11]),1)
maxval[12] = round(float(maxval[12]),2)

In [11]:
table1p = PrettyTable()

# Set table style
table1p.title = "TABLE 1"
table1p.border = True
table1p.header = True
table1p.align = 'c'

rows = []
table1p.field_names = ["VARIABLES", "Mean", "Median", "St. Dev.", "Min", "Max", "Obs."]

variables = ["Panel A. City demographics","Fr. all immigrants","Fr. recent immigrants","Recent immigrants over\n1900 population","City population (1,000s)","Panel B. Economic outcomes","Employed","Log occupational scores","Value added per establishment","Establishment size","Panel C. Political outcomes","Tax rate per 1,000$\nof assessed valuation","Expenditures per capita","Democrats’ vote share","DW nominate score"]
i = 0
for elem in variables:
    list_name = f"row{i}"
    globals()[list_name] = []
    rows.append(globals()[list_name])
    globals()[list_name].append(elem)
    globals()[list_name].append(mean[i])
    globals()[list_name].append(median[i])
    globals()[list_name].append(stdev[i])
    globals()[list_name].append(minval[i])
    globals()[list_name].append(maxval[i])
    globals()[list_name].append(obs[i])
    i += 1

for elem in rows:
    table1p.add_row(elem)
    
table1p.align["VARIABLES"] = 'l'

In [12]:
table2p = PrettyTable()

# Set table style
table2p.title = "TABLE 2"
table2p.border = True
table2p.header = True
table2p.align = 'c'

f_left = [f"{val:<10}" for val in f]

table2p.field_names = ["Dep. variable: fraction of immigrants", "(1)", "(2)", "(3)", "(4)", "(5)", "(6)"]
table2p.add_rows([coeffs,se,pop1900,predpop,msa,log1900,f_left,cities,nobs])

table2p.align["Dep. variable: fraction of immigrants"] = 'l'

In [13]:
print(table1p)
print()
print(table2p)
print("*** p < 0.01 ** p < 0.05 * p < 0.1")

+-----------------------------------------------------------------------------------+
|                                      TABLE 1                                      |
+-------------------------------+-------+--------+----------+--------+-------+------+
| VARIABLES                     |  Mean | Median | St. Dev. |  Min   |  Max  | Obs. |
+-------------------------------+-------+--------+----------+--------+-------+------+
| Panel A. City demographics    |       |        |          |        |       |      |
| Fr. all immigrants            | 0.152 | 0.149  |  0.097   | 0.007  | 0.518 | 540  |
| Fr. recent immigrants         | 0.042 | 0.026  |  0.044   | 0.001  | 0.343 | 540  |
| Recent immigrants over        | 0.073 | 0.048  |  0.078   | 0.002  | 0.677 | 540  |
| 1900 population               |       |        |          |        |       |      |
| City population (1,000s)      | 190.1 | 76.05  |  509.9   | 30.20  |  6930 | 540  |
| Panel B. Economic outcomes    |       |        |    